In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchaudio
from torchaudio.transforms import Spectrogram, AmplitudeToDB
from torch import nn

# Paramètres
fs = 16000  # Fréquence d'échantillonnage
duration = 10  # Durée totale de l'audio en secondes
c = 343  # Vitesse du son en m/s

# Distance entre les microphones
distance_between_mics = 2  # Distance 

# Simuler deux sources sonores
source1_freq = 5000  # Fréquence de la première personne (en Hz)
source2_freq = 1500  # Fréquence de la deuxième personne (en Hz)

# Générer le signal de bruit ambiant avec amplitude réduite
noise = torch.normal(mean=0, std=0.1, size=(duration * fs,))
# Générer les signaux pour les deux sources pendant 2 secondes (de 4 à 6 secondes)
t_source = torch.linspace(0, 4, 4 * fs)
source1 = torch.sin(2 * np.pi * source1_freq * t_source)
source2 = torch.sin(2 * np.pi * source2_freq * t_source)

# Insérer les signaux des sources dans le bruit ambiant
signal = noise.clone()
signal[2*fs:6*fs] += source1
signal[2*fs:6*fs] += source2

# Ajouter des délais pour simuler l'arrivée des sons aux microphones
delay1 = distance_between_mics / c  # Délai en secondes pour la première source
delay_samples1 = int(delay1 * fs)  # Conversion du délai en nombre d'échantillons
delay2 = 3 * distance_between_mics / c  # Délai pour la deuxième source
delay_samples2 = int(delay2 * fs)

# Générer les signaux capturés par chaque microphone
mic1 = signal 
mic2 = torch.roll(signal, shifts=delay_samples1) + signal
mic3 = torch.roll(signal, shifts=delay_samples2) + torch.roll(signal, shifts=delay_samples2)
# Calculer les spectrogrammes complexes avec PyTorch
transform = Spectrogram(n_fft=128, hop_length=64, power=None)
spec1 = transform(mic1.unsqueeze(0))
spec2 = transform(mic2.unsqueeze(0))
spec3 = transform(mic3.unsqueeze(0))

# Calcul du Logarithm Power Spectrum (LPS)
amplitude_to_db = AmplitudeToDB()
lps1 = amplitude_to_db(spec1.abs())
lps2 = amplitude_to_db(spec2.abs())
lps3 = amplitude_to_db(spec3.abs())


# Obtenir les temps et les fréquences
times = np.linspace(0, duration, spec1.size(-1))
frequencies = torch.linspace(0, fs // 2, spec1.size(-2))

# Tracer les spectrogrammes d'amplitude et de phase
plt.figure(figsize=(12, 12))

plt.subplot(3, 1, 1)
plt.plot(t_source.numpy(), mic1[2*fs:6*fs].numpy())
plt.title('Signal reçu par le microphone 1')
plt.xlabel('Temps [sec]')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 2)
plt.plot(t_source.numpy(), mic2[2*fs:6*fs].numpy())
plt.title('Signal reçu par le microphone 2')
plt.xlabel('Temps [sec]')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 3)
plt.plot(t_source.numpy(), mic3[2*fs:6*fs].numpy())
plt.title('Signal reçu par le microphone 3')
plt.xlabel('Temps [sec]')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 12))

plt.subplot(3, 2, 1)
plt.pcolormesh(times, frequencies, lps1[0].numpy(), shading='gouraud')
plt.title('Spectrogramme d\'Amplitude - Microphone 1')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Amplitude [dB]')

print(times.shape)
print(frequencies.numpy().shape)
print(lps1[0].numpy().shape)

plt.subplot(3, 2, 3)
plt.pcolormesh(times, frequencies, lps2[0].numpy(), shading='gouraud')
plt.title('Spectrogramme d\'Amplitude - Microphone 2')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Amplitude [dB]')

plt.subplot(3, 2, 5)
plt.pcolormesh(times, frequencies, lps3[0].numpy(), shading='gouraud')
plt.title('Spectrogramme d\'Amplitude - Microphone 3')
plt.xlabel('Temps [sec]')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Amplitude [dB]')

plt.subplot(3, 2, 2)
plt.pcolormesh(times, frequencies, torch.angle(spec1[0]).numpy(), shading='gouraud')
plt.title('Spectrogramme de Phase - Microphone 1')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Phase [radians]')

plt.subplot(3, 2, 4)
plt.pcolormesh(times, frequencies, torch.angle(spec2[0]).numpy(), shading='gouraud')
plt.title('Spectrogramme de Phase - Microphone 2')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Phase [radians]')

plt.subplot(3, 2, 6)
plt.pcolormesh(times, frequencies, torch.angle(spec3[0]).numpy(), shading='gouraud')
plt.title('Spectrogramme de Phase - Microphone 3')
plt.xlabel('Temps [sec]')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Phase [radians]')

plt.tight_layout()
plt.show()

In [ ]:
# Calculer l'Inter-Channel Phase Difference (IPD) pour les paires de microphones
ipd12 = torch.angle(spec1) - torch.angle(spec2)
ipd13 = torch.angle(spec1) - torch.angle(spec3)
ipd23 = torch.angle(spec2) - torch.angle(spec3)




# Tracer les IPD
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.pcolormesh(times, frequencies, ipd12[0].numpy(), shading='gouraud')
plt.title('IPD - Microphone 1 et 2')
plt.xlabel('Temps [sec]')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Phase [radians]')

plt.subplot(1, 3, 2)
plt.pcolormesh(times, frequencies, ipd13[0].numpy(), shading='gouraud')
plt.title('IPD - Microphone 1 et 3')
plt.xlabel('Temps [sec]')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Phase [radians]')

plt.subplot(1, 3, 3)
plt.pcolormesh(times, frequencies, ipd23[0].numpy(), shading='gouraud')
plt.title('IPD - Microphone 2 et 3')
plt.xlabel('Temps [sec]')
plt.ylabel('Fréquence [Hz]')
plt.colorbar(label='Phase [radians]')

plt.tight_layout()
plt.show()



In [ ]:


# Calcul de l'Angle Feature (AF)
def compute_af(theta, frequencies, distance_between_mics, c):
    delta_m = distance_between_mics
    v_m_theta = 2 * np.pi * frequencies * delta_m * np.cos(theta) / c
    return v_m_theta

# Directions spécifiées {0, 0.5π, π, 1.5π}
theta_values = [0, 0.5 * np.pi, np.pi, 1.5 * np.pi]



# Redimensionner les fréquences pour correspondre aux dimensions des IPD
frequencies = torch.linspace(0, fs // 2, spec1.size(-2)).unsqueeze(-1)

# Calculer et tracer les AFθ pour chaque direction
plt.figure(figsize=(12, 16))

for i, theta in enumerate(theta_values):
    v_m_theta = compute_af(theta, frequencies, distance_between_mics, c).reshape(1, -1, 1)
    af1 = torch.cos(v_m_theta - ipd12)
    af2 = torch.cos(v_m_theta - ipd13)
    af3 = torch.cos(v_m_theta - ipd23)

    plt.subplot(4, 3, 3*i + 1)
    plt.pcolormesh(np.arange(spec1.size(-1)), frequencies.squeeze(), af1[0].numpy(), shading='gouraud')
    plt.title(f'AFθ - Microphone 1 et 2 - θ={theta} rad')
    plt.ylabel('Fréquence [Hz]')
    plt.colorbar(label='Amplitude')

    plt.subplot(4, 3, 3*i + 2)
    plt.pcolormesh(np.arange(spec2.size(-1)), frequencies.squeeze(), af2[0].numpy(), shading='gouraud')
    plt.title(f'AFθ - Microphone 1 et 3 - θ={theta} rad')
    plt.ylabel('Fréquence [Hz]')
    plt.colorbar(label='Amplitude')

    plt.subplot(4, 3, 3*i + 3)
    plt.pcolormesh(np.arange(spec3.size(-1)), frequencies.squeeze(), af3[0].numpy(), shading='gouraud')
    plt.title(f'AFθ - Microphone 2 et 3 - θ={theta} rad')
    plt.xlabel('Temps [sec]')
    plt.ylabel('Fréquence [Hz]')
    plt.colorbar(label='Amplitude')

plt.tight_layout()
plt.show()